In [5]:
import logging, json, os, requests, datetime
import azure.functions as func
from azure.servicebus import ServiceBusClient, ServiceBusMessage
from dotenv import load_dotenv

In [2]:
output = {"key": "", "file_path": "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/rest-api/layout.png", "model": "prebuilt-layout",
          "Operation-location": "https://formr-pdf.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/e7ece836-f3fc-4231-a959-f177ff454de4?api-version=2022-08-31"}
type(output)

dict

In [1]:
Operation_location = "https://formr-pdf.cognitiveservices.azure.com/formrecognizer/documentModels/prebuilt-layout/analyzeResults/2e5e5a55-cc90-494b-9ee3-892321dad26b?api-version=2022-08-31"

In [3]:
msg = json.dumps(output, indent=8) # dict to string
data = json.loads(msg)

In [4]:
load_dotenv('..')
cog_endpoint = os.getenv('COGNITIVE_SERVICE_FORM_RECOGNIZER_ENDPOINT')
cog_key = os.getenv('COGNITIVE_SERVICE_FORM_RECOGNIZER_KEY')
model= "prebuilt-layout"
form_recognizer_url = f"{cog_endpoint}formrecognizer/documentModels/{model}:analyze?api-version=2022-08-31"

In [9]:
r = requests.get(
    #data('Operation-location'), 
    Operation_location,
    headers={'Ocp-Apim-Subscription-Key': cog_key}
    )
r

<Response [200]>

In [16]:
if r.json()('status') == 'succeeded':
        data['results'] = r.json()
        print('data.results fields'+ str(data.keys()))
elif r.json()['status'] in ('notStarted', 'running'):
        pass

res=data['results']['analyzeResult']['tables']
print("document-layout API resulted in 'result'.'analyzeResult : "+ str(data['results']['analyzeResult'].keys()))

print("found" , len(res), 'tables')

data.results fieldsdict_keys(['key', 'file_path', 'model', 'Operation-location', 'results'])
document-layout API resulted in 'result'.'analyzeResult : dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'tables', 'paragraphs', 'styles'])
found 4 tables


In [17]:
import process_data
if model=='prebuilt-layout':
        tables=()
        tables=process_data.process_tables(data)
        processed_data=tables
if model == 'prebuilt-invoice':
        invoice=()
        invoice = process_data.process_invoice(data)
        processed_data=invoice

In [19]:
for t in processed_data['value']:
    print(t('headers'))

['Date of Birth:', '23 February 1974']
['Must be completed\nInduction Medication Complete if applicable', 'Drug Name', 'Dose', 'Presentation Required', 'Route', 'Frequency']
['Maintenance Medication', 'Drug Name please use brand names', 'Dose', 'Presentation Required', 'Route', 'Frequency']
['Qualification/GMC:', 'RN/NMC 81G 0175E', 'Telephone: 01226 432387']


In [27]:
import pandas as pd

t= processed_data['value'][0]
r=t['raw_text']
h=t['headers']
df = pd.DataFrame([r[i:i+len(h)] for i in range(0, len(r), len(h))],columns=t['headers'])
df

,Date of Birth:,23 February 1974
0,Hospital No:,141714
1,NHS No:,408 323
2,Weight (kg):,


## Pushing to index

In [31]:
body = {  
  "value": [  
    {  
      "@search.action": "merge",  
      "metadata_storage_path": "aHR0cHM6Ly9mcnN0b3JhZ2UyMDIzLmJsb2IuY29yZS53aW5kb3dzLm5ldC9zY2llbnN1cy9SeCUyMDYucG5n0",
      "tables":["table content 1 API call from VSC"]
    },
    {  
      "@search.action": "merge",  
      "metadata_storage_path": "aHR0cHM6Ly9mcnN0b3JhZ2UyMDIzLmJsb2IuY29yZS53aW5kb3dzLm5ldC9zY2llbnN1cy9SeDEwLnRpZg2",
      "tables":["table content 2 API call from VSC"]
    }
    
  ]  
} 

In [32]:
load_dotenv('..')
search_endpoint = os.getenv('AZURE_SEARCH_ENDPOINT') 
search_key = os.getenv('AZURE_SEARCH_ENDPOINT_KEY')
index_name = os.getenv('AZURE_SEARCH_INDEX_NAME') 



search_endpoint = "https://search-fr.search.windows.net"
search_key = "nvp3H9mYvjimcqxtvzE1FXP9kkc5HfXHSUOjyaEQ9yAzSeC6AxRC"
index_name = "demo2023-fr"

print(search_endpoint)
print(search_key)
print(index_name)

https://search-fr.search.windows.net
nvp3H9mYvjimcqxtvzE1FXP9kkc5HfXHSUOjyaEQ9yAzSeC6AxRC
demo2023-fr


In [34]:
headers = {'Content-Type': 'application/json', 'api-key': search_key}
r = requests.post(search_endpoint + f"/indexes/{index_name}/docs/index?api-version=2019-05-06", headers=headers, json=body)
r

<Response [200]>

In [29]:
r = requests.get(search_endpoint + f"/indexes/{index_name}/docs/?search=R201028226&api-version=2019-05-06", headers=headers)
r.content

b'{"@odata.context":"https://search-fr.search.windows.net/indexes(\'demo2023-fr\')/$metadata#docs(*)","value":[{"@search.score":0.6735479,"metadata_storage_path":"aHR0cHM6Ly9mcnN0b3JhZ2UyMDIzLmJsb2IuY29yZS53aW5kb3dzLm5ldC9zY2llbnN1cy9SeCUyMDYucG5n0","metadata_storage_path_decoded":"https://frstorage2023.blob.core.windows.net/sciensus/Rx%206.png","path":"https://frstorage2023.blob.core.windows.net/sciensus/Rx%206.png","metadata_storage_size":163182,"metadata_storage_last_modified":"2023-02-08T10:47:22Z","metadata_storage_name":"Rx 6.png","merged_content":" Sciensus Private and Confidential . R Amgevita\\u00ae (adalimumab) Homecare Prescription *** NHS Service *** 27/10 Hospital Name Western General Hospital Prescription type Hospital Address Crewe Road South, Edinburgh New patient & EH4 2XU Renewal Post Code Dose change Therapy Switch Patient Name (including title) Mrs Ann Smith Address & 123 Somewhere Cold Post Code Edinburgh Date of Birth: 27/07/1967 Blueteq Ref: n/a Hospital Number: 

In [23]:
body = {'value': 
[{
    '@search.action': 'merge', 
    'metadata_storage_path': 'aHR0cHM6Ly9mcnN0b3JhZ2UyMDIzLmJsb2IuY29yZS53aW5kb3dzLm5ldC9zY2llbnN1cy9SeCUyMDYucG5n0', 
    'tables': ["{'page_number': 1, 'raw_count': 5, 'column_count': 2, 'headers': ['Date of Birth:', '23 February 1974'], 'cells': [{'text': 'Date of Birth:', 'rowIndex': 0, 'colIndex': 0, 'is_header': False}, {'is_header': True}, {'text': '23 February 1974', 'rowIndex': 0, 'colIndex': 1, 'is_header': False}, {'is_header': True}, {'text': 'Hospital No:', 'rowIndex': 1, 'colIndex': 0, 'is_header': False}, {'text': '141714', 'rowIndex': 1, 'colIndex': 1, 'is_header': False}, {'text': 'NHS No:', 'rowIndex': 2, 'colIndex': 0, 'is_header': False}, {'text': '408 323', 'rowIndex': 2, 'colIndex': 1, 'is_header': False}, {'text': 'Weight (kg):', 'rowIndex': 3, 'colIndex': 0, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 1, 'is_header': False}], 'raw_text': []}, {'page_number': 1, 'raw_count': 3, 'column_count': 5, 'headers': ['Must be completed\\nInduction Medication Complete if applicable', 'Drug Name', 'Dose', 'Presentation Required', 'Route', 'Frequency'], 'cells': [{'text': 'Must be completed\\nInduction Medication Complete if applicable', 'rowIndex': 0, 'colIndex': 0, 'is_header': False}, {'is_header': True}, {'text': 'Drug Name', 'rowIndex': 1, 'colIndex': 0, 'is_header': False}, {'is_header': True}, {'text': 'Dose', 'rowIndex': 1, 'colIndex': 1, 'is_header': False}, {'is_header': True}, {'text': 'Presentation Required', 'rowIndex': 1, 'colIndex': 2, 'is_header': False}, {'is_header': True}, {'text': 'Route', 'rowIndex': 1, 'colIndex': 3, 'is_header': False}, {'is_header': True}, {'text': 'Frequency', 'rowIndex': 1, 'colIndex': 4, 'is_header': False}, {'is_header': True}, {'text': '', 'rowIndex': 2, 'colIndex': 0, 'is_header': False}, {'text': '', 'rowIndex': 2, 'colIndex': 1, 'is_header': False}, {'text': '', 'rowIndex': 2, 'colIndex': 2, 'is_header': False}, {'text': '', 'rowIndex': 2, 'colIndex': 3, 'is_header': False}, {'text': '', 'rowIndex': 2, 'colIndex': 4, 'is_header': False}], 'raw_text': []}, {'page_number': 1, 'raw_count': 6, 'column_count': 5, 'headers': ['Maintenance Medication', 'Drug Name please use brand names', 'Dose', 'Presentation Required', 'Route', 'Frequency'], 'cells': [{'text': 'Maintenance Medication', 'rowIndex': 0, 'colIndex': 0, 'is_header': False}, {'is_header': True}, {'text': 'Drug Name please use brand names', 'rowIndex': 1, 'colIndex': 0, 'is_header': False}, {'is_header': True}, {'text': 'Dose', 'rowIndex': 1, 'colIndex': 1, 'is_header': False}, {'is_header': True}, {'text': 'Presentation Required', 'rowIndex': 1, 'colIndex': 2, 'is_header': False}, {'is_header': True}, {'text': 'Route', 'rowIndex': 1, 'colIndex': 3, 'is_header': False}, {'is_header': True}, {'text': 'Frequency', 'rowIndex': 1, 'colIndex': 4, 'is_header': False}, {'is_header': True}, {'text': 'Benepall (Etanercept)', 'rowIndex': 2, 'colIndex': 0, 'is_header': False}, {'text': '50mg', 'rowIndex': 2, 'colIndex': 1, 'is_header': False}, {'text': 'PEN', 'rowIndex': 2, 'colIndex': 2, 'is_header': False}, {'text': 'SC', 'rowIndex': 2, 'colIndex': 3, 'is_header': False}, {'text': '1/52', 'rowIndex': 2, 'colIndex': 4, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 0, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 1, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 2, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 3, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 4, 'is_header': False}, {'text': '', 'rowIndex': 4, 'colIndex': 0, 'is_header': False}, {'text': '', 'rowIndex': 4, 'colIndex': 1, 'is_header': False}, {'text': '', 'rowIndex': 4, 'colIndex': 2, 'is_header': False}, {'text': '', 'rowIndex': 4, 'colIndex': 3, 'is_header': False}, {'text': '', 'rowIndex': 4, 'colIndex': 4, 'is_header': False}, {'text': 'Ancillaries required Complete if applicable', 'rowIndex': 5, 'colIndex': 0, 'is_header': False}, {'text': '', 'rowIndex': 5, 'colIndex': 2, 'is_header': False}], 'raw_text': []}, {'page_number': 1, 'raw_count': 6, 'column_count': 7, 'headers': ['Qualification/GMC:', 'RN/NMC 81G 0175E', 'Telephone: 01226 432387'], 'cells': [{'text': 'Qualification/GMC:', 'rowIndex': 0, 'colIndex': 0, 'is_header': False}, {'is_header': True}, {'text': 'RN/NMC 81G 0175E', 'rowIndex': 0, 'colIndex': 1, 'is_header': False}, {'is_header': True}, {'text': 'Telephone: 01226 432387', 'rowIndex': 0, 'colIndex': 4, 'is_header': False}, {'is_header': True}, {'text': 'Clinical Screened By Pharmacy: Please indicate', 'rowIndex': 1, 'colIndex': 0, 'is_header': False}, {'text': '× Yes\\n:selected:', 'rowIndex': 1, 'colIndex': 2, 'is_header': False}, {'text': '0 No\\n:unselected:', 'rowIndex': 1, 'colIndex': 3, 'is_header': False}, {'text': 'Date:', 'rowIndex': 1, 'colIndex': 5, 'is_header': False}, {'text': '31/10/2', 'rowIndex': 1, 'colIndex': 6, 'is_header': False}, {'text': 'Pharmacist Signature:', 'rowIndex': 2, 'colIndex': 0, 'is_header': False}, {'text': '', 'rowIndex': 2, 'colIndex': 1, 'is_header': False}, {'text': '', 'rowIndex': 2, 'colIndex': 2, 'is_header': False}, {'text': '', 'rowIndex': 2, 'colIndex': 3, 'is_header': False}, {'text': '', 'rowIndex': 2, 'colIndex': 4, 'is_header': False}, {'text': 'Telephone:', 'rowIndex': 2, 'colIndex': 5, 'is_header': False}, {'text': '5046', 'rowIndex': 2, 'colIndex': 6, 'is_header': False}, {'text': 'Pharmacist Name: Please print', 'rowIndex': 3, 'colIndex': 0, 'is_header': False}, {'text': 'A.', 'rowIndex': 3, 'colIndex': 1, 'is_header': False}, {'text': 'Watson', 'rowIndex': 3, 'colIndex': 2, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 3, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 4, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 5, 'is_header': False}, {'text': '', 'rowIndex': 3, 'colIndex': 6, 'is_header': False}, {'text': 'Qualification/Gphc:', 'rowIndex': 4, 'colIndex': 0, 'is_header': False}, {'text': '5101622.', 'rowIndex': 4, 'colIndex': 1, 'is_header': False}, {'text': '', 'rowIndex': 4, 'colIndex': 3, 'is_header': False}, {'text': '', 'rowIndex': 4, 'colIndex': 4, 'is_header': False}, {'text': '', 'rowIndex': 4, 'colIndex': 5, 'is_header': False}, {'text': '', 'rowIndex': 4, 'colIndex': 6, 'is_header': False}, {'text': 'Purchase Order No: If applicable', 'rowIndex': 5, 'colIndex': 0, 'is_header': False}, {'text': '27239', 'rowIndex': 5, 'colIndex': 2, 'is_header': False}, {'text': '', 'rowIndex': 5, 'colIndex': 3, 'is_header': False}, {'text': '', 'rowIndex': 5, 'colIndex': 4, 'is_header': False}, {'text': '', 'rowIndex': 5, 'colIndex': 5, 'is_header': False}, {'text': '', 'rowIndex': 5, 'colIndex': 6, 'is_header': False}], 'raw_text': []}"]
    }]
    }
